# Procesar los datos de los videos

Este notebook se encarga de transformar los datos crudos provenientes de archivos de video en un formato estructurado y útil para el análisis. Parte de archivos que almacenan la posicion tridimensional de distintos puntos del cuerpo, después calcula el ángulo entre disintos puntos del cuepro, y por último se realizan cálculos estadísticos sobre los ángulos

**Objetivo**
*  Leer y procesar los archivos crudos para crear un DataFrame que contenga información esencial sobre cada grabación, incluyendo detalles sobre el paciente, el número de repetición, el ID del movimiento, la precisión del gesto (si es correcto o no), y la posición de los keypoints.
* Crear un segundo DataFrame que contenga los ángulos calculados entre algunos de los keypoints, lo que ayudará a entender mejor la postura y el movimiento durante la ejecución del gesto.
* Sobre este último, realizar cálculos estadísticos sobre los ángulos calculados entre keypoints para cada repetición y gesto. 

**Entrada**
* Archivos de datos de videos dentro del directorio ``SkeletonData/RawData``, que contienen información sobre el paciente, las repeticiones, los IDs de movimiento, y las posiciones de keypoints.

**Salida**
* ``Resultados/raw_pacientes.csv``: contiene información detallada sobre cada grabación. Es el archivo de entrada de ``analisis_datos.ipynb``.
* ``Resulados/angles.csv``: incluye los ángulos calculados entre keypoints, facilitando el análisis de la postura y el movimiento durante el gesto. Este será el archivo de entrada de ``medidas_angulos.ipynb`` y de ``analisis_datos.ipynb``.
* ``Resultados/medidasPerRepetition.csv``: contiene una fila por repetición y gesto, que incluye estadísticas para cada ángulo calculado.

**Índice**
1. [Procesar Raw Data Folder](#1-procesar-raw-data-folder)\
    1.1 [Datos de los nombres de los archivos](#11-datos-de-los-nombres-de-los-archivos)\
    1.2 [Datos de los archivos](#12-datos-de-los-archivos)
2. [Calcular los ángulos](#2-calcular-los-ángulos)
3. [Cálculos estadísticos sobre los ángulos](#3-cálculos-estadísticos-sobre-los-ángulos)


In [7]:
# importar librerias necesarias
import pandas as pd # para manejar dataframes
import numpy as np
import functions as functions
import os

# Para ignorar los FutureWarning
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

*****
## 1. Procesar Raw Data Folder

El conjunto de datos incluye a 30 sujetos, de los cuales 14 son sanos y 16 son pacientes. Cada participante realiza 9 tipos de ejercicios, con 6 repeticiones de cada uno.

La carpeta contiene 2598 archivos de texto con información de los videos, donde cada archivo corresponde a un sujeto realizando una repetición de un gesto. Hay información tanto en el contenido del archivo como en el propio nombre del archivo.

* En el nombre del archivo se encuentran los identificadores del sujeto, del día, del gesto, el número de repetición, si es correcto y la posición del sujeto durante el ejercicio.

* Dentro de cada archivo hay información sobre las coordenadas de 25 puntos del cuerpo captados por un sensor que graba a 30 fps. Por lo tanto, por cada segundo de video, se generan 30 líneas en el archivo de texto. Cada línea sigue el siguiente formato: marca de tiempo, XX, XX, seguido de 25 pares de (NombreArticulación, EstadoSeguimiento, coordenada 3D X, coordenada 3D Y, coordenada 3D Z, coordenada 2D X, coordenada 2D Y).


### 1.1 Datos de los nombres de los archivos

Los nombre de los archivos siguen el siguiente formato: *SubjectID_DateID_GestureLabel_RepetitionNumber_CorrectLabel_Position.txt*

* *SubjectID*: id uniquely identifying the person performing the exercise
* *DateID*: id identifying the session in which the person was recorded
* *GestureLabel*: Label identifying the gesture; possible values are from 0 to 8
* *RepetitionNumber*: Each gesture was repeated several times and this shows the repetition number
* *CorrectLabel*: A value of 1 represents a gesture labeled as being correctly executed, while a value of 2 is for a gesture labeled as incorrect
* *Position*: Some of the persons performed the gestures sitting on a chair or wheelchair, while others standing

En el archivo `functions.py` se ha creado la siguiente función:
<small>
```python
def leer_nombre_archivo(archivo:str) -> list[str]:
    """
    Extrae datos específicos del nombre de un archivo de texto.

    Parámetros
    ----------
    archivo : str
        Nombre del archivo, con el formato
        'SubjectID_DateID_GestureLabel_RepetitionNumber_CorrectLabel_Position.txt'.

    Return
    -------
    campos : list[str]
        Lista de cadenas de texto que contiene los datos extraídos del nombre del archivo:
        [SubjectID, DateID, GestureLabel, RepetitionNumber, CorrectLabel, Position].
    """
    archivo = archivo.split('.')[0] # quita la extension txt
    campos = archivo.split('_') # separa los campos por _
    return campos
```
</small>


### 1.2 Datos de los archivos
Each raw data file contains per line: timestamp, XX, XX, followed by a 25 pairs of (JointName, TrackedStatus, 3d coordinate X, 3d coordinate Y, 3d coordinate Z, 2d coordinate X, 2d coordinate Y)

In [8]:
# Función para extraer la información de dentro de los archivos
def leer_datos_archivo(directorio:str, columnas:list[str]) -> pd.DataFrame:
    """
    Compila la información de los archivos en un directorio y los guarda en un DataFrame.

    Parámetros
    ----------
    directorio : str
        Nombre del directorio donde se encuentran los archivos.
    columnas : list[str]
        Lista con los nombres de las columnas para el DataFrame de salida.

    Return
    -------
    pd.DataFrame
        DataFrame con todos los datos recopilados de los archivos.
    """
    # Crea una lista con los nombres de los archivos en el directorio
    file_list = os.listdir(directorio)

    # lista para almacenar los datos extraídos
    list_data = []

    # Itera sobre cada archivo
    for file_name in file_list:
         # Extrae los datos del nombre del archivo
        campos = functions.leer_nombre_archivo(file_name)

        with open(os.path.join(directorio, file_name), 'r') as file:
            for line in file:
                 # Divide la línea por comas y extrae la información deseada
                 # omitiendo timestamp y otros datos innecesarios
                line_data = line.strip().split(',')[3:]
                # Quita los paréntesis
                cleaned_data = [item.replace('(', '').replace(')', '') for item in line_data] 
                # por cada linea de los archivos necesitamos bloques de 7 valores
                for i in range(0, len(cleaned_data), 7): 
                    list_data.append(campos + cleaned_data[i:i + 7])
    df = pd.DataFrame(list_data, columns=columnas)
    return df

In [9]:
# directorio donde se encuentran los datos
directory = '../dataset/SkeletonData/RawData' 

columnas = ['SubjectID', 'DateID', 'GestureLabel', 'RepetitionNumber', 'CorrectLabel', 'Position',
            'JointName', 'TrackedStatus', '3D_X', '3D_Y', '3D_Z', '2D_X', '2D_Y']

# Extraer la informacion y almacenarla en un DataFrame
df_data = leer_datos_archivo(directory, columnas)

In [10]:
# Eliminar columnas innceserarias
df_data.drop(['TrackedStatus', 'DateID', 'Position', '2D_X', '2D_Y'], axis=1, inplace=True)

# Añadir columna más descriptiva de los gestos
gesture_mapping = {
    '0': 'EFL',
    '1': 'EFR',
    '2': 'SFL',
    '3': 'SFR',
    '4': 'SAL',
    '5': 'SAR',
    '6': 'SFE',
    '7': 'STL',
    '8': 'STR'
}

# Nueva columna basada en el mapeo
columna_gestrure_name = df_data['GestureLabel'].map(gesture_mapping)

# Insertar la columna al lado de GestureLabel
df_data.insert(2, 'GestureName', columna_gestrure_name)

df_data

,SubjectID,GestureLabel,GestureName,RepetitionNumber,CorrectLabel,JointName,3D_X,3D_Y,3D_Z
0,102,0,EFL,10,1,SpineBase,-0.08088344,-0.2248836,2.661578
1,102,0,EFL,10,1,SpineMid,-0.06806522,0.04492111,2.628779
2,102,0,EFL,10,1,Neck,-0.055614,0.3082771,2.583972
3,102,0,EFL,10,1,Head,-0.04478608,0.4328104,2.593495
4,102,0,EFL,10,1,ShoulderLeft,-0.2232155,0.2021449,2.549825
...,...,...,...,...,...,...,...,...,...
5500495,307,8,STR,9,1,SpineShoulder,-0.05799517,0.5291457,2.422904
5500496,307,8,STR,9,1,HandTipLeft,-0.302538,-0.1131345,2.284269
5500497,307,8,STR,9,1,ThumbLeft,-0.2783904,-0.06298634,2.269769
5500498,307,8,STR,9,1,HandTipRight,0.1525867,-0.136378,2.45287


In [11]:
# guardar los datos en formato csv
df_data.to_csv('../Resultados/raw_pacientes.csv', index=False)

******
## 2. Calcular los ángulos

<div style="text-align: center;">
<img src="../Imagenes/gestures.png" width="500"/>
</div>

In [12]:
def apply_angles(df: pd.DataFrame) -> pd.DataFrame:
    # Aplica la función para caluclar los ángulos a los datos en crudo

    angles = []

    # Agrupar el DataFrame por cada 25 filas e iterar
    for _, group in df.groupby(np.arange(len(df)) // 25):
        # Calculate angles for the group
        elbow_angle_left = functions.calculate_angle(group, 'ShoulderLeft', 'ElbowLeft', 'WristLeft')
        elbow_angle_right = functions.calculate_angle(group, 'ShoulderRight', 'ElbowRight', 'WristRight')
        left_arm_angle = functions.calculate_angle(group, 'HipLeft', 'ShoulderLeft', 'ElbowLeft')
        right_arm_angle = functions.calculate_angle(group, 'HipRight', 'ShoulderRight', 'ElbowRight')
        arms_together_angle = functions.calculate_angle(group, 'SpineBase', 'SpineShoulder', 'WristLeft')
        wrist_angle_left = functions.calculate_angle(group, 'ElbowLeft', 'WristLeft', 'HandLeft')
        shoulder_angle_left = functions.calculate_angle(group, 'ShoulderLeft', 'SpineShoulder', 'ElbowLeft')
        wrist_angle_right = functions.calculate_angle(group, 'ElbowRight', 'WristRight', 'HandRight')
        shoulder_angle_right = functions.calculate_angle(group, 'ShoulderRight', 'SpineShoulder', 'ElbowRight')
        hip_angle_left = functions.calculate_angle(group, 'HipLeft', 'SpineBase', 'KneeLeft')
        knee_angle_left = functions.calculate_angle(group, 'HipLeft', 'KneeLeft', 'AnkleLeft')
        ankle_angle_left = functions.calculate_angle(group, 'KneeLeft', 'AnkleLeft', 'FootLeft')
        hip_angle_right = functions.calculate_angle(group, 'HipRight', 'SpineBase', 'KneeRight')
        knee_angle_right = functions.calculate_angle(group, 'HipRight', 'KneeRight', 'AnkleRight')
        ankle_angle_right = functions.calculate_angle(group, 'KneeRight', 'AnkleRight', 'FootRight')
        
        # # Extraer columnas adicionales
        additional_data = group.iloc[0][['SubjectID', 'GestureLabel', 'GestureName', 'RepetitionNumber', 'CorrectLabel']]

        # Almacenar la información en un diccionario
        angles.append({
            **additional_data,
            'ElbowAngleLeft': elbow_angle_left,
            'ElbowAngleRight': elbow_angle_right,
            'ShoulderAngleLeft': shoulder_angle_left,
            'ShoulderAngleRight': shoulder_angle_right,
            'WristAngleLeft': wrist_angle_left,
            'WristAngleRight': wrist_angle_right,
            'HipAngleLeft': hip_angle_left,
            'KneeAngleLeft': knee_angle_left,
            'AnkleAngleLeft': ankle_angle_left,
            'HipAngleRight': hip_angle_right,
            'KneeAngleRight': knee_angle_right,
            'AnkleAngleRight': ankle_angle_right,
            'LeftArmAngle': left_arm_angle,
            'RightArmAngle': right_arm_angle,
            'ArmsTogetherAngle': arms_together_angle
        })

    # Crear un DataFrame a partir de la lista de diccionarios
    return pd.DataFrame(angles)

In [13]:
# Aplica la función para caluclar los ángulos a los datos en crudo
df_angles = apply_angles(df_data)
# Visualizar el dataframe
df_angles

,SubjectID,GestureLabel,GestureName,RepetitionNumber,CorrectLabel,ElbowAngleLeft,ElbowAngleRight,ShoulderAngleLeft,ShoulderAngleRight,WristAngleLeft,WristAngleRight,HipAngleLeft,KneeAngleLeft,AnkleAngleLeft,HipAngleRight,KneeAngleRight,AnkleAngleRight,LeftArmAngle,RightArmAngle,ArmsTogetherAngle
0,102,0,EFL,10,1,36.445861,36.582917,148.276141,150.538111,4.831462,13.634312,132.747369,99.561246,79.739535,143.350817,105.108123,81.345980,149.337083,146.077955,142.905759
1,102,0,EFL,10,1,36.501008,37.026046,148.254429,150.517838,16.758538,1.784125,132.648786,99.678021,80.376922,143.683790,105.217669,81.391368,149.402103,145.995028,143.001279
2,102,0,EFL,10,1,36.431684,37.239103,148.233151,150.577629,17.130884,7.832669,132.684043,99.996248,82.609605,143.639654,105.216908,81.310906,149.368710,145.908696,142.965117
3,102,0,EFL,10,1,36.645339,68.564605,148.323425,151.666569,6.310039,30.207149,132.864128,99.998782,82.691058,142.434449,102.783610,82.313890,149.327745,145.480989,142.880512
4,102,0,EFL,10,1,36.704133,67.595392,148.257312,156.530739,5.198054,48.424470,132.908134,99.745980,80.982213,142.287131,102.197980,82.432998,149.420443,139.564886,142.876385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220015,307,8,STR,9,1,19.235168,16.366695,142.466122,142.949282,10.597409,8.590306,107.710008,17.152761,57.808079,94.948037,1.709497,41.616584,156.103380,158.651430,149.864559
220016,307,8,STR,9,1,19.337504,16.460773,142.614506,142.959204,9.097187,8.083043,107.352914,16.479229,57.733793,94.653231,1.263963,41.879291,156.056261,158.714378,149.734213
220017,307,8,STR,9,1,19.762179,16.292120,142.579738,142.903745,9.288112,10.157666,107.398951,16.368690,58.670153,94.346427,1.041109,78.704138,156.130164,158.836560,149.712017
220018,307,8,STR,9,1,20.339779,16.195526,142.685324,142.897308,7.335038,9.108455,107.436579,16.163032,57.798046,94.144821,1.015447,78.752737,156.114139,158.911807,149.496620


In [14]:
# guardar los datos en formato csv
df_angles.to_csv('../Resultados/angles.csv', index=False)

******
## 3. Cálculos estadísticos sobre los ángulos

Realizar cálculos estadísticos sobre los ángulos calculados entre keypoints para cada repetición y gesto. 

In [15]:
# Cálculos estadísticos sobre los ángulos
def calculos_estadísticos(df:pd.DataFrame) -> pd.DataFrame:
    """
    Realiza cálculos estadísticos sobre los ángulos en un
    DataFrame agrupado por sujeto, gesto y número de repetición.

    Parámetros
    ----------
    df : pd.DataFrame
        DataFrame que contiene información sobre los ángulos,
        así como otras columnas relacionadas con el sujeto,
        gesto, repetición, etc.

    Returns
    -------
    pd.DataFrame
        DataFrame que contiene las estadísticas descriptivas
        calculadas para cada grupo de ángulos, con una fila por
        combinación de sujeto, gesto y repetición.
    """

    # Agrupa el DataFrame 
    groups = df.groupby(["SubjectID", "GestureLabel", "RepetitionNumber"])

    # Lista para almacenar los datos de salida
    data = []

    # Itera sobre cada grupo
    for (subject_id, gesture_label, repetition_number), group in groups:
        # Selecciona solo las columnas que contienen los ángulos 
        angles = group.iloc[:, 6:]

        # Calcula la media y la desviación estándar para los ángulos
        means = angles.mean()
        std_devs = angles.std()

        # Almacena las estadísticas en un diccionario 
        data.append({
            'SubjectID': subject_id,
            'GestureLabel': gesture_label,
            'GestureName': group['GestureName'].iloc[0],
            'RepetitionNumber': repetition_number,
            'CorrectLabel': group['CorrectLabel'].iloc[0],
            'Duration': len(group),  # Duración en número de frames
            'standardDeviation': std_devs,
            'Maximum': angles.max(),
            'Minimum': angles.min(),
            'Mean': means,
            'Range': angles.max() - angles.min(),
            'Variance': angles.var(),
            'CoV': std_devs / means,  # Coeficiente de variación
            'Skewness': angles.skew(),  # Asimetría
            'Kurtosis': angles.kurtosis()  # Curtosis
        })

    # Convierte la lista de diccionarios en un DataFrame y lo ordena
    df_stats = pd.DataFrame(data)
    df_stats = df_stats.sort_values(['SubjectID', 'GestureLabel', 'RepetitionNumber'])

    return df_stats

In [16]:
# Cálculos sobre los ángulos
df_stats = calculos_estadísticos(df_angles)
df_stats.head()

,SubjectID,GestureLabel,GestureName,RepetitionNumber,CorrectLabel,Duration,standardDeviation,Maximum,Minimum,Mean,Range,Variance,CoV,Skewness,Kurtosis
0,102,0,EFL,1,1,74,ElbowAngleRight 53.294387 ShoulderAngleL...,ElbowAngleRight 163.526254 ShoulderAngle...,ElbowAngleRight 16.729038 ShoulderAngle...,ElbowAngleRight 82.879290 ShoulderAngle...,ElbowAngleRight 146.797215 ShoulderAngle...,ElbowAngleRight 2840.291699 ShoulderAngl...,ElbowAngleRight 0.643036 ShoulderAngleLe...,ElbowAngleRight 0.172849 ShoulderAngleLe...,ElbowAngleRight -1.490457 ShoulderAngleL...
1,102,0,EFL,10,1,30,ElbowAngleRight 43.928490 ShoulderAngleL...,ElbowAngleRight 158.100220 ShoulderAngle...,ElbowAngleRight 36.582917 ShoulderAngle...,ElbowAngleRight 93.041786 ShoulderAngle...,ElbowAngleRight 121.517303 ShoulderAngle...,ElbowAngleRight 1929.712218 ShoulderAngl...,ElbowAngleRight 0.472137 ShoulderAngleLe...,ElbowAngleRight 0.205721 ShoulderAngleLe...,ElbowAngleRight -1.545249 ShoulderAngleLe...
2,102,0,EFL,11,1,46,ElbowAngleRight 45.611381 ShoulderAngleL...,ElbowAngleRight 158.425536 ShoulderAngle...,ElbowAngleRight 38.097913 ShoulderAngle...,ElbowAngleRight 91.932549 ShoulderAngle...,ElbowAngleRight 120.327623 ShoulderAngle...,ElbowAngleRight 2080.398042 ShoulderAngl...,ElbowAngleRight 0.496140 ShoulderAngleLe...,ElbowAngleRight 0.143495 ShoulderAngleLe...,ElbowAngleRight -1.638852 ShoulderAngleLe...
3,102,0,EFL,12,1,49,ElbowAngleRight 42.200983 ShoulderAngleL...,ElbowAngleRight 150.741507 ShoulderAngle...,ElbowAngleRight 34.982253 ShoulderAngle...,ElbowAngleRight 76.088864 ShoulderAngle...,ElbowAngleRight 115.759254 ShoulderAngle...,ElbowAngleRight 1780.922975 ShoulderAngl...,ElbowAngleRight 0.554628 ShoulderAngleLe...,ElbowAngleRight 0.775270 ShoulderAngleLe...,ElbowAngleRight -1.089126 ShoulderAngleL...
4,102,0,EFL,2,1,64,ElbowAngleRight 49.408774 ShoulderAngleL...,ElbowAngleRight 159.540915 ShoulderAngle...,ElbowAngleRight 15.502762 ShoulderAngle...,ElbowAngleRight 91.406754 ShoulderAngle...,ElbowAngleRight 144.038153 ShoulderAngle...,ElbowAngleRight 2441.226976 ShoulderAngl...,ElbowAngleRight 0.540537 ShoulderAngleLe...,ElbowAngleRight -0.155463 ShoulderAngleLe...,ElbowAngleRight -1.431169 ShoulderAngleL...


Como se ve en la cabecera del DataFrame anterior es poco legible, por lo que se va a formatear para mejorar la legibilidad del DataFrame y evitar la representación diccionarios dentro de las columnas del DataFrame

Se he creado una función para formatear las columnas en:

<small>

```python

def formatear_columnas(df:pd.DataFrame, nombre_columna:str) -> pd.DataFrame: 
    '''
    Esta función toma una columna de un DataFrame que contiene diccionarios
    y expande cada diccionario en nuevas columnas, donde cada nueva columna
    corresponde a:  clave del diccionario + _ + nombre de la columna original. 

    Parámetro
    --------
    nombre_columna : str
        Nombre de la columna del DataFrame que contiene los diccionarios.

    Return
    --------
     Un nuevo DataFrame con columnas separadas para cada clave de los diccionarios.
    '''
    keys = set().union(*(d.keys() for d in df[nombre_columna]))
    data = {}
    for key in keys:
        data[key + '_' + nombre_columna] = df[nombre_columna].apply(lambda x: x.get(key))
    return pd.DataFrame(data)

```

</small>

In [17]:
columnas = ['standardDeviation', 'Maximum', 'Minimum', 'Mean', 'Range',
            'Variance', 'CoV', 'Skewness', 'Kurtosis']
# Reformatea cada columna de diccionario y concatena los resultados
nuevas_columnas = pd.concat([functions.formatear_columnas(df_stats[col], col) for col in columnas], axis=1)

# Concatenar las nuevas columnas con el DataFrame original
df = pd.concat([df_stats, nuevas_columnas], axis=1)

# Elimina las columnas originales que contenían diccionarios
df = df.drop(columnas, axis=1)

# Ordena el DataFrame 
## pasamos a numerico para que ordene bien los numeros
df = df.apply(pd.to_numeric, errors='ignore')

df = df.sort_values(['SubjectID', 'GestureLabel', 'RepetitionNumber'])

# Mostrar el resultado
df

,SubjectID,GestureLabel,GestureName,RepetitionNumber,CorrectLabel,Duration,KneeAngleLeft_standardDeviation,WristAngleLeft_standardDeviation,KneeAngleRight_standardDeviation,ShoulderAngleRight_standardDeviation,...,WristAngleRight_Kurtosis,ElbowAngleRight_Kurtosis,AnkleAngleLeft_Kurtosis,LeftArmAngle_Kurtosis,RightArmAngle_Kurtosis,ArmsTogetherAngle_Kurtosis,HipAngleRight_Kurtosis,AnkleAngleRight_Kurtosis,HipAngleLeft_Kurtosis,ShoulderAngleLeft_Kurtosis
0,102,0,EFL,1,1,74,0.483290,1.377540,0.302958,3.294714,...,0.375607,-1.490457,-0.921846,-0.988983,0.221473,-0.851102,-1.213044,15.481424,0.195086,-1.251144
4,102,0,EFL,2,1,64,0.558593,1.627189,0.324361,4.742960,...,1.154400,-1.431169,-1.415922,-1.293845,-1.486033,-1.419682,0.166379,30.599465,-0.788811,-0.472467
5,102,0,EFL,3,1,58,0.682435,1.698975,0.542524,5.070928,...,9.590802,-1.448128,-1.203117,-1.244715,-0.791329,-1.562968,-1.304679,2.346517,-0.915824,-0.808711
6,102,0,EFL,4,1,60,0.614214,2.016340,0.352648,4.030705,...,4.133624,-1.585881,-1.515334,-1.533188,0.202151,-1.690368,0.005324,7.285595,-1.185226,-1.177012
7,102,0,EFL,5,1,63,0.489771,1.700859,0.259491,5.021403,...,5.640903,-1.460889,-0.517289,-1.258867,0.837403,-1.719744,-0.484635,15.865257,-0.592432,-0.689188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480,307,8,STR,8,1,48,6.503301,3.335282,1.206067,0.311175,...,0.056064,0.234936,0.312861,-0.781945,-1.313588,-0.339300,-1.014746,-0.270964,-1.222994,-0.971195
2481,307,8,STR,9,1,48,6.647659,3.502306,1.591213,0.314987,...,-1.006323,-0.759991,1.056307,-1.644095,-1.551620,-0.863515,-1.651462,-1.671159,-1.672822,-0.963412
2471,307,8,STR,10,1,47,5.285898,4.508933,1.502639,0.299210,...,-0.776532,-0.707586,-0.851119,-0.826005,-1.482760,-0.449516,-1.103517,-2.061997,-1.403915,0.037584
2472,307,8,STR,11,1,51,6.237513,2.577782,1.153539,0.189624,...,-0.052683,-0.899253,-0.156794,-0.620023,-1.347955,-0.445218,-1.697380,-1.011064,-1.462976,-0.660478


In [18]:
# Guardar el DataFrame en formato csv
df.to_csv('../Resultados/medidasPerRepetition.csv', index=False)